This notebook contains some experiments.
It does not contain important code.

In [ ]:
import numpy as np
import torch, torchvision
import torch.nn as nn
from torchvision import datasets
import torchvision.transforms as transforms
import matplotlib.pyplot as plt 
from PIL import Image
from models.SimpleUpscale import SimpleUpscale
from models.SimpleConv import SimpleConv
from train import train_model
from dataclass.ImageClass import UpscaledImages
from time import time
from torch.utils.data import Dataset
from torchvision.transforms import ToTensor, Resize, Compose


In [ ]:
# transform = transforms.Compose([transforms.Resize((32, 32)), 
#                                 transforms.ToTensor(), 
#                                 transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]) 

# trainset = datasets.CIFAR10(root = "./data", train=True, download=True, transform=transform)

# trainloader = torch.utils.data.DataLoader(trainset, batch_size=100, shuffle=True, num_workers=2)

In [ ]:
# Simple conv test
# device = "cuda:0" if torch.cuda.is_available() else "cpu"
# model = SimpleConv().to(device)
# learning_rate=0.01
# optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
# criterion = nn.MSELoss()
# train_model(model=model, optimizer=optimizer, criterion=criterion, trainloader=trainloader, epochs=5, device=device)

In [ ]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"


In [ ]:
class MicroImages(Dataset):
    def __init__(self, root_dir):
        """Initializes a dataset containing images and labels."""
        super().__init__()
        self.root_dir = root_dir
        self.transform_lr = Compose([ToTensor(), Resize((32, 32))])
        self.transform_hr = Compose([ToTensor(), Resize((64, 64))])

        self.data = []
        for i in range(800):
            image_hr_dir = self.root_dir + 'DIV2K_train_HR/' + "{:0>4}".format(i+1) + '.png'
            image_lr_dir = self.root_dir + 'DIV2K_train_LR_bicubic/X2/' + "{:0>4}x2".format(i+1) + '.png'
            image_hr = Image.open(image_hr_dir)
            image_lr = Image.open(image_lr_dir)
            self.data.append((self.transform_lr(image_lr), self.transform_hr(image_hr)))

        self.size=800

    def __len__(self):
        """Returns the size of the dataset."""
        return len(self.data)

    def __getitem__(self, index):
        """Returns the index-th data item of the dataset."""

        return self.data[index]

In [ ]:
# Data set
# def collate(batch):
#     images_lr = [item[0] for item in batch]
#     images_hr = [item[1] for item in batch]
#     return [images_lr, images_hr]

# trouver un pre processsing, reduire taille image ?, crop ? resize ? 
# Les images doivent resize (idéalement la plus grande possible)
# Refaire un dataset ?

# fonctions de vizualization

#  Gerer les batch_size, mini_batch ? -> backward seulement 

transform_lr = transforms.Compose([transforms.ToTensor(), 
                                torch.nn.ZeroPad2d(128),
                                transforms.CenterCrop(128),
                                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]) 
transform_hr = transforms.Compose([transforms.ToTensor(),
                                torch.nn.ZeroPad2d(256),
                                transforms.CenterCrop(256)
                                ]) 
trainset = MicroImages("../data/")


In [ ]:
torch.cuda.empty_cache()

In [ ]:
class SimpleUpscale(nn.Module):
    def __init__(self):
        super(SimpleUpscale, self).__init__()

        #upsamplin vs conv transpose

        # self.net = nn.Sequential(     
        #              nn.Conv2d(3,   12, kernel_size=3,  stride=1, padding=1),   nn.BatchNorm2d(12), nn.LeakyReLU(),
        #              nn.Conv2d(12,  12, kernel_size=3,  stride=1, padding=1),   nn.BatchNorm2d(12), nn.LeakyReLU(),
        #              nn.Conv2d(12,  48, kernel_size=3,  stride=1, padding=1),   nn.BatchNorm2d(48), nn.LeakyReLU(),
        #              nn.Conv2d(48,  48, kernel_size=3,  stride=1, padding=1),   nn.BatchNorm2d(48), nn.LeakyReLU(),
        #              nn.Conv2d(48,  48, kernel_size=3,  stride=1, padding=1),   nn.BatchNorm2d(48), nn.LeakyReLU(),
        #     nn.Conv2d(48,  48, kernel_size=3,  stride=1, padding=1),   nn.BatchNorm2d(48), nn.LeakyReLU(),
        #     nn.Conv2d(48,  48, kernel_size=3,  stride=1, padding=1),   nn.BatchNorm2d(48), nn.LeakyReLU(),
        #     nn.Upsample(scale_factor=2, mode="bicubic"), nn.BatchNorm2d(48), nn.LeakyReLU(),
        #     nn.Conv2d(48,  48, kernel_size=3,  stride=1, padding=1),   nn.BatchNorm2d(48), nn.LeakyReLU(),
        #     nn.Conv2d(48,  48, kernel_size=3,  stride=1, padding=1),   nn.BatchNorm2d(48), nn.LeakyReLU(),
        #     nn.Conv2d(48,  48, kernel_size=3,  stride=1, padding=1),   nn.BatchNorm2d(48), nn.LeakyReLU(),
        #     nn.Conv2d(48,  48, kernel_size=3,  stride=1, padding=1),   nn.BatchNorm2d(48), nn.LeakyReLU(),
        #     nn.Conv2d(48,  48, kernel_size=3,  stride=1, padding=1),   nn.BatchNorm2d(48), nn.LeakyReLU(),
        #     nn.Conv2d(48,  48, kernel_size=3,  stride=1, padding=1),   nn.BatchNorm2d(48), nn.LeakyReLU(),
        #     nn.Conv2d(48,  48, kernel_size=3,  stride=1, padding=1),   nn.BatchNorm2d(48), nn.LeakyReLU(),
        #     nn.Conv2d(48,  48, kernel_size=3,  stride=1, padding=1),   nn.BatchNorm2d(48), nn.LeakyReLU(),
        #     nn.Conv2d(48,  12, kernel_size=3,  stride=1, padding=1),              nn.BatchNorm2d(12), nn.LeakyReLU(),
        #              nn.Conv2d(12,  12, kernel_size=3,  stride=1, padding=1),   nn.BatchNorm2d(12), nn.LeakyReLU(),
        #              nn.Conv2d(12,   3, kernel_size=3,  stride=1, padding=1),   nn.BatchNorm2d(3), nn.LeakyReLU(),
        #              nn.Conv2d(3,    3, kernel_size=3,  stride=1, padding=1))
            
        self.net2 = nn.Sequential(     
            nn.Conv2d(3,   48, kernel_size=3,  stride=2, padding=1),   nn.BatchNorm2d(48), nn.LeakyReLU(),
            nn.Conv2d(48,   48, kernel_size=3,  stride=1, padding=1),   nn.BatchNorm2d(48), nn.LeakyReLU(), # opt
            nn.Conv2d(48,   96, kernel_size=3,  stride=2, padding=1),   nn.BatchNorm2d(96), nn.LeakyReLU(), #opt
            nn.Conv2d(96,   96, kernel_size=3,  stride=1, padding=1),   nn.BatchNorm2d(96), nn.LeakyReLU(), # opt
            nn.Upsample(scale_factor=2, mode="bicubic"), nn.BatchNorm2d(96), nn.LeakyReLU(), #opt
            nn.Conv2d(96,   96, kernel_size=3,  stride=1, padding=1),   nn.BatchNorm2d(96), nn.LeakyReLU(), # opt
            nn.Upsample(scale_factor=2, mode="bicubic"), nn.BatchNorm2d(96), nn.LeakyReLU(),
            nn.Conv2d(96,   48, kernel_size=3,  stride=1, padding=1),   nn.BatchNorm2d(48), nn.LeakyReLU(),
            nn.Upsample(scale_factor=2, mode="bicubic"), nn.BatchNorm2d(48), nn.LeakyReLU(),
            nn.Conv2d(48,   48, kernel_size=3,  stride=1, padding=1),   nn.BatchNorm2d(48), nn.LeakyReLU(),
            nn.Conv2d(48,  48, kernel_size=3,  stride=1, padding=1),   nn.BatchNorm2d(48), nn.LeakyReLU(),
            nn.Conv2d(48,  48, kernel_size=3,  stride=1, padding=1),   nn.BatchNorm2d(48), nn.LeakyReLU(),
            nn.Conv2d(48,  48, kernel_size=3,  stride=1, padding=1),   nn.BatchNorm2d(48), nn.LeakyReLU(),
            nn.Conv2d(48,  48, kernel_size=3,  stride=1, padding=1),   nn.BatchNorm2d(48), nn.LeakyReLU(),
            nn.Conv2d(48,  48, kernel_size=3,  stride=1, padding=1),   nn.BatchNorm2d(48), nn.LeakyReLU(),
            nn.Conv2d(48,  48, kernel_size=3,  stride=1, padding=1),   nn.BatchNorm2d(48), nn.LeakyReLU(),
            nn.Conv2d(48,  48, kernel_size=3,  stride=1, padding=1),   nn.BatchNorm2d(48), nn.LeakyReLU(),
            nn.Conv2d(48,  48, kernel_size=3,  stride=1, padding=1),   nn.BatchNorm2d(48), nn.LeakyReLU(),
            nn.Conv2d(48,  48, kernel_size=3,  stride=1, padding=1),   nn.BatchNorm2d(48), nn.LeakyReLU(),
            nn.Conv2d(48,  48, kernel_size=3,  stride=1, padding=1),   nn.BatchNorm2d(48), nn.LeakyReLU(),
            nn.Conv2d(48,  48, kernel_size=3,  stride=1, padding=1),   nn.BatchNorm2d(48), nn.LeakyReLU(),
            nn.Conv2d(48,  12, kernel_size=3,  stride=1, padding=1),              nn.BatchNorm2d(12), nn.LeakyReLU(),
                     nn.Conv2d(12,  12, kernel_size=3,  stride=1, padding=1),   nn.BatchNorm2d(12), nn.LeakyReLU(),
                     nn.Conv2d(12,   3, kernel_size=3,  stride=1, padding=1),   nn.BatchNorm2d(3), nn.LeakyReLU(),
                     nn.Conv2d(3,    3, kernel_size=3,  stride=1, padding=1))

    def forward(self, x):
        return self.net2(x)

In [ ]:
trainloader = torch.utils.data.DataLoader(trainset, batch_size=20, shuffle=True)

In [ ]:
model = SimpleUpscale().to(device)
learning_rate=0.0005
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.MSELoss()

In [ ]:
print(torch.cuda.memory_summary())

In [ ]:
# Training
test = False
epochs = 16

start = time()
epoch_train_losses = []
for i in range(epochs):
    tmp_loss = []
    for (x, y) in trainloader: # [batch_size x 3 x w x h]
        outputs = model(x.to(device))
        loss = criterion(outputs, y.to(device))
        tmp_loss.append(loss.detach())
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        x.detach()
        y.detach()
    epoch_train_losses.append(torch.tensor(tmp_loss).mean())
    print(f"Epoch {i+1}")

end = time()
print(f"Training took {end - start} seconds for {epochs} epochs, or {(end - start)/epochs} seconds per epochs")

torch.cuda.empty_cache()



In [ ]:
plt.plot(range(epochs), epoch_train_losses)

In [ ]:
# hr_size = []
# lr_size = []
# for image_lr, image_hr in dataset:
#     hr_size.append(image_hr.size())
#     lr_size.append(image_hr.size())

In [ ]:
image_lr, image_hr = trainset[0]

In [ ]:
image_lr.size()

In [ ]:
image_lr_cropped = transforms.CenterCrop((800, 800))(torch.nn.ZeroPad2d(800)(image_lr))
image_lr_cropped.size()

In [ ]:
def show_images(img):
    img = img 
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

In [ ]:
show_images(torchvision.utils.make_grid([image_lr]))


In [ ]:
show_images(torchvision.utils.make_grid([(image_hr)]))


In [ ]:
# from dataclass.ImageClass import UpscaledImages
# reload(UpscaledImages)
# import 

ouput_hr = model(image_hr.unsqueeze(0).to(device))

In [ ]:
show_images(torchvision.utils.make_grid(ouput_hr.cpu().detach()))


In [ ]:
show_images(torchvision.utils.make_grid(list(map(lambda x: x[0], trainset[0:5]))))

GAN: 
Generator :
Sequence de bloque convolutionel, upsampling, Sequence de bloque convolutionel

Discriminator :
Convolution jusqua arriver a un output scalaire. [0, 1]

In [ ]:
show_images(torchvision.utils.make_grid(list(map(lambda x: model(x[0].unsqueeze(0).to(device))[0].detach().to("cpu"), trainset[0:5]))))

In [ ]:
torch.cuda.empty_cache()

In [ ]:
rec_image = model(image_hr.unsqueeze(0).to(device))
rec_image = rec_image.to(device)
rec_image = model(rec_image)
rec_image = model(rec_image)
show_images(torchvision.utils.make_grid(rec_image.cpu().detach()))

In [ ]:
mem_params = sum([param.nelement()*param.element_size() for param in model.parameters()])
mem_bufs = sum([buf.nelement()*buf.element_size() for buf in model.buffers()])
mem = mem_params + mem_bufs # in bytes
print(f"model is {mem} bytes, {mem/(1024**2)} megabytes")

Autoencoder

In [ ]:
import numpy as np
import torch, torchvision
import torch.nn as nn
from torchvision import datasets
import torchvision.transforms as transforms
import matplotlib.pyplot as plt 
from PIL import Image

In [ ]:
def show_images(img):
    img = img 
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

In [ ]:
transform = transforms.Compose([transforms.Resize((32, 32)),
                                transforms .ToTensor(),
                                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]) # write the sequence of appropriate transformations
trainset = datasets.CIFAR10(root="./data", train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4, shuffle=True, num_workers=2)
